In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [ ]:
df=pd.read_csv('/content/pwd.csv',on_bad_lines='skip')

In [ ]:
df.head()

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1


In [ ]:
X=df['password']
Y=df['strength']

In [ ]:
X.size

669640

In [ ]:
Y.size

669640

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector=TfidfVectorizer(analyzer='char',max_features=50)

In [ ]:
X.isnull().sum()

1

In [ ]:
X = X.fillna('')


In [ ]:
vector.fit(X)

TfidfVectorizer(analyzer='char', max_features=50)

In [ ]:
X_trans=vector.transform(X)

In [ ]:
X_trans=X_trans.toarray()

In [ ]:
X_trans

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.33592566],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33760963,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.64376663, 0.        ,
        0.        ]])

In [ ]:
X_fet=vector.get_feature_names_out()

In [ ]:
X_fet

array([' ', '!', '#', '$', '%', '&', '*', '+', '-', '.', '/', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '?', '@', '_', 'a', 'b',
       'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o',
       'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype=object)

In [ ]:
X_proc=pd.DataFrame(X_trans,columns=X_fet)

In [ ]:
X_proc.head()

,,!,#,$,%,&,*,+,-,.,...,q,r,s,t,u,v,w,x,y,z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.335926
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.289451,0.297311,0.0,0.0,0.392465,0.0,0.0,0.337610,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.390622,0.448306
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.293943,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X_proc,Y)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
regressor=LogisticRegression(multi_class='ovr')

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],   # Regularization terms
    'C': [0.01, 0.1, 1, 10, 100],                   # Inverse of regularization strength (higher values reduce regularization)
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],  # Solvers that handle different penalties
    'class_weight': [None, 'balanced'],              # Adjusting weights for unbalanced classes
    'l1_ratio': [0, 0.25, 0.5, 0.75, 1],             # Only used when penalty is elasticnet
    'max_iter': [100, 200, 300, 500],                # Maximum number of iterations
}

In [ ]:
random_search = RandomizedSearchCV(estimator=regressor,
                                   param_distributions=param_grid,
                                   n_iter=100,           # Number of random combinations to try
                                   cv=3,                # Cross-validation splitting strategy
                                   verbose=2,           # Verbosity level
                                   random_state=42,     # Reproducibility
                                   n_jobs=-1)

In [ ]:
regressor.fit(x_train,y_train)

LogisticRegression(multi_class='ovr')

In [ ]:
y_pred=regressor.predict(x_test)

In [ ]:
y_proab=regressor.predict_proba(x_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8122095454273939
[[  4096  18224     22]
 [  2686 118568   2897]
 [    74   7535  13308]]
              precision    recall  f1-score   support

           0       0.60      0.18      0.28     22342
           1       0.82      0.96      0.88    124151
           2       0.82      0.64      0.72     20917

    accuracy                           0.81    167410
   macro avg       0.75      0.59      0.63    167410
weighted avg       0.79      0.81      0.78    167410



In [ ]:
y_inp=input()
X_inp=vector.transform([y_inp]).toarray()
X_inp_fet=vector.get_feature_names_out()
x_inp_proc=pd.DataFrame(X_inp,columns=X_inp_fet)
y_pred_inp=regressor.predict(X_inp_proc)
print(y_pred_inp)

ghghttn3


NameError: name 'X_inp_proc' is not defined

In [ ]:
y_inp=input()
X_inp=vector.transform([y_inp]).toarray()
X_inp_fet=vector.get_feature_names_out()
x_inp_proc=pd.DataFrame(X_inp,columns=X_inp_fet)
y_pred_inp=regressor.predict(x_inp_proc) # changed X_inp_proc to x_inp_proc
if y_pred_inp[0]==0:
  print('Password is weak')
elif y_pred_inp[0]==1:
  print('Password is medium')
else:
  print('Password is strong')

eghrighevorwovbegberof
Password is medium


In [ ]:
import pickle as pkl


In [ ]:
pkl.dump(regressor,open('model.pkl','wb'))
pkl.dump(vector,open('vector.pkl','wb'))